In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.content, "lxml")

In [5]:
table = soup.find_all('table')[0]

In [72]:
canada = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'], index = list(range(len(table.find_all('tr')))))

In [73]:
canada.shape

(290, 3)

In [74]:
import ipdb
from ipdb import set_trace

In [75]:
row_marker = 0
for i, row in enumerate(table.find_all('tr')):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        canada.iat[i,column_marker] = column.get_text()
        column_marker += 1

In [76]:
canada.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [77]:
canada.dropna(subset=['Borough'], inplace=True)

In [78]:
canada.shape

(289, 3)

In [79]:
canada = canada[~canada['Borough'].str.contains('Not assigned')]

In [80]:
canada.shape

(212, 3)

In [81]:
temp = canada.groupby(by='Postcode')['Neighbourhood'].apply(','.join).reset_index()

In [82]:
temp['Neighbourhood'] = temp['Neighbourhood'].str.replace("\n", "")

In [83]:
canada = canada[['Postcode', 'Borough']]

In [84]:
canada = pd.merge(canada, temp, how='inner', on=['Postcode'])

In [90]:
canada.drop_duplicates(inplace=True)

In [92]:
canada.head(n=20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge,Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens,Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson,Garden District"


In [93]:
canada.shape

(103, 3)

In [94]:
canada.to_csv('./canada_postarea.csv', index=False)